In [1]:
%load_ext autoreload
%autoreload 2

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict, OrderedDict

from utils.generic import init_params
from models.seir.seir_testing import SEIR_Testing
from data.processing import get_district_time_series
from data.dataloader import get_covid19india_api_data

## Load covid19 data

In [3]:
dataframes = get_covid19india_api_data()

In [4]:
dataframes.keys()

dict_keys(['df_tested', 'df_statewise', 'df_india_time_series', 'df_districtwise', 'df_raw_data', 'df_raw_data_2', 'df_travel_history', 'df_resources'])

In [5]:
df_district = get_district_time_series(dataframes)

In [6]:
df_district

,date,total_infected
0,2020-03-09,1
1,2020-03-10,4
2,2020-03-11,4
3,2020-03-12,5
4,2020-03-13,5
5,2020-03-14,5
6,2020-03-15,5
7,2020-03-16,5
8,2020-03-17,8
9,2020-03-18,11


## Loss Calculation Functions

In [7]:
def _calc_rmse(y_pred, y_true, log=True):
    if log:
        y_true = np.log(y_true)
        y_pred = np.log(y_pred)
    loss = np.sqrt(np.mean((y_true - y_pred)**2))
    return loss

def _calc_mape(y_pred, y_true):
    y_pred = y_pred[y_true > 0]
    y_true = y_true[y_true > 0]

    ape = np.abs((y_true - y_pred + 0) / y_true) *  100
    loss = np.mean(ape)
    return loss

def calc_loss_dict(states_time_matrix, df, method='rmse', rmse_log=False):
    pred_hospitalisations = states_time_matrix[6] + states_time_matrix[7] + states_time_matrix[8]
    pred_recoveries = states_time_matrix[9]
    pred_fatalities = states_time_matrix[10]
    pred_infectious_unknown = states_time_matrix[2] + states_time_matrix[4]
    pred_total_cases = pred_hospitalisations + pred_recoveries + pred_fatalities
    
    if method == 'rmse':
        if rmse_log:
            calculate = lambda x, y : _calc_rmse(x, y)
        else:
            calculate = lambda x, y : _calc_rmse(x, y, log=False)
    
    if method == 'mape':
            calculate = lambda x, y : _calc_mape(x, y)
    
    losses = {}
#     losses['hospitalised'] = calculate(pred_hospitalisations, df['Hospitalised'])
#     losses['recovered'] = calculate(pred_recoveries, df['Recovered'])
#     losses['fatalities'] = calculate(pred_fatalities, df['Fatalities'])
#     losses['active_infections'] = calculate(pred_infectious_unknown, df['Active Infections (Unknown)'])
    losses['total'] = calculate(pred_total_cases, df['total_infected'])
    
    return losses

def calc_loss(states_time_matrix, df, method='rmse', rmse_log=False):
    losses = calc_loss_dict(states_time_matrix, df, method, rmse_log)
#     loss = losses['hospitalised'] + losses['recovered'] + losses['total'] + losses['active_infections']
    loss = losses['total']
    return loss

## Initialize params and state values

In [8]:
vanilla_params, testing_params, state_init_values = init_params()

In [9]:
vanilla_params

{'R0': 2.2,
 'T_trans': 1.3181818181818181,
 'T_inc': 5.2,
 'T_inf': 2.9,
 'T_recov_mild': 11.1,
 'T_recov_severe': 28.6,
 'T_hosp': 5,
 'T_death': 32,
 'P_mild': 0.78,
 'P_severe': 0.2,
 'P_fatal': 0.02,
 'intervention_day': 100,
 'lockdown_removal_day': 45,
 'intervention_amount': 0.33,
 'N': 7000000.0}

In [10]:
state_init_values

OrderedDict([('S', 0.9999998571428571),
             ('E', 0),
             ('I', 1.4285714285714285e-07),
             ('D_E', 0),
             ('D_I', 0),
             ('R_mild', 0),
             ('R_severe_home', 0),
             ('R_severe_hosp', 0),
             ('R_fatal', 0),
             ('C', 0),
             ('D', 0)])

## Set priors for parameters of interest

In [23]:
## assuming uniform priors, following dictionary contains the ranges
prior_ranges = OrderedDict()
prior_ranges['R0'] = (1.6, 3)
prior_ranges['T_inc'] = (4, 5)
prior_ranges['T_inf'] = (3, 4)
prior_ranges['T_recov_severe'] = (9, 20)
prior_ranges['P_severe'] = (0.3, 0.99)
prior_ranges['intervention_amount'] = (0.3, 1)


## Proposal function to sample theta_new given theta_old

In [24]:
## stddevs for all params are set to 1 now. must tune later.
proposal_sigmas = OrderedDict()
proposal_sigmas['R0'] = 1
proposal_sigmas['T_inc'] = 1
proposal_sigmas['T_inf'] = 1
proposal_sigmas['T_recov_severe'] = 5
proposal_sigmas['P_severe'] = 1
proposal_sigmas['intervention_amount'] = 1

def proposal(theta_old):
    theta_new = np.random.normal(loc=theta_old, scale=proposal_sigmas.values())
    return theta_new